In [ ]:
# Import required modules and functions.
import os
import pandas as pd
import random
import time
from datetime import datetime, timedelta

from utils import (
    get_times, calc_coherence, run_coherence, get_max_corr, get_frame_files, 
    get_strain_data, find_max_corr_channel, plot_max_corr_chan, give_group_v2
)
from gwpy.timeseries import TimeSeries
import plotly.express as px
import plotly

# Set parameters manually (instead of using argparse).
# Change these parameters as needed:
lowfreq = 55         # Lower frequency bound (Hz)
highfreq = 65        # Upper frequency bound (Hz)
dur = 900            # Duration of data in seconds
ifo = "K1"           # Interferometer
# For example, set savedir to a folder whose name is a GPS time, e.g., "K1_automatic/1370183890/"
savedir = os.path.join(os.getcwd(), "K1_automatic", "1370183890")


print("Parameters:")
print("  lowfreq =", lowfreq)
print("  highfreq =", highfreq)
print("  dur =", dur)
print("  ifo =", ifo)
print("  savedir =", savedir)

In [ ]:
# Robustly extract a numeric time from the savedir.
savedir_abs = os.path.abspath(savedir)
time_str = os.path.basename(os.path.normpath(savedir_abs))
if not time_str.isdigit():
    time_str = os.path.basename(os.path.dirname(savedir_abs))
if not time_str.isdigit():
    raise ValueError(f"Could not extract a numeric time from savedir '{savedir}'")
time_val = int(time_str)
print("Current time is:", time_val)

In [ ]:
from gwpy.segments import Segment, SegmentList
from gwpy.time import to_gps

# Set the data segment. You can use a date string or a GPS time.
# Here we use the date from the timestamp extracted (you could also set it manually).
# For example, if you have a date string, you can do:
# date_str = "2023-01-01"  (for example) and then:
# start_gps = to_gps(datetime.strptime(date_str, '%Y-%m-%d'))
# For simplicity, we'll use the numeric time as the start_gps.
start_gps = time_val  
end_gps = start_gps + dur
seg_list = SegmentList([Segment(start_gps, end_gps)])
times_segs = get_times(seg_list, duration=dur)
print("Generated time stamps from segment:", times_segs)

In [ ]:
# Load channel lists (voltage, OMC, and mic channels).
channel_path = os.path.join('/home/shu-wei.yeh/coherence-monitor/channel_files', ifo)
volt_chans = pd.read_csv(os.path.join(channel_path, 'volt_channels.csv'), header=None, names=['channel'])
# omc_chans = pd.read_csv(os.path.join(channel_path, 'omc_channels.csv'), header=None, names=['channel'])
mic_chans = pd.read_csv(os.path.join(channel_path, 'mic_channels.csv'), header=None, names=['channel'])

print("Voltage channels:")
print(volt_chans.head(), '\n')

print("OMC channels:")
# print(omc_chans.head(), '\n')

print("Mic channels:")
print(mic_chans.head())

In [ ]:
# Randomly choose one GPS start time from the available times.
time_chosen = random.choice(times_segs)
print("Chosen GPS start time:", time_chosen)

In [ ]:
# Load the strain data for the interval [time_chosen, time_chosen + dur).
ht_data = get_strain_data(time_chosen, time_chosen + dur, dur, ifo=ifo)
if ht_data is None:
    raise RuntimeError("Failed to load h(t) data.")
print("Strain data loaded successfully.")

In [ ]:
import multiprocessing

def get_coherence_volt(channel_list, ifo, t0, strain_data, savedir, duration):
    files_ = get_frame_files(t0, t0 + duration, duration, ifo=ifo, directory='/data/KAGRA/raw/full/')
    print("Got {} witness file(s) for VOLT channels".format(len(files_)))
    if not files_:
        raise ValueError("No frame files found.")
    run_coherence(
        channel_list=channel_list,
        frame_files=files_,
        starttime=t0,
        endtime=t0 + duration,
        ifo=ifo,
        strain_data=strain_data,
        savedir=savedir
    )

def get_coherence_omc(channel_list, ifo, t0, strain_data, savedir, duration):
    files_ = get_frame_files(t0, t0 + duration, duration, ifo=ifo, directory='/data/KAGRA/raw/full/')
    print("Got {} witness file(s) for OMC channels".format(len(files_)))
    if not files_:
        raise ValueError("No frame files found.")
    run_coherence(
        channel_list=channel_list,
        frame_files=files_,
        starttime=t0,
        endtime=t0 + duration,
        ifo=ifo,
        strain_data=strain_data,
        savedir=savedir
    )

def get_coherence_mic(channel_list, ifo, t0, strain_data, savedir, duration):
    files_ = get_frame_files(t0, t0 + duration, duration, ifo=ifo, directory='/data/KAGRA/raw/full/')
    print("Got {} witness file(s) for MIC channels".format(len(files_)))
    if not files_:
        raise ValueError("No frame files found.")
    run_coherence(
        channel_list=channel_list,
        frame_files=files_,
        starttime=t0,
        endtime=t0 + duration,
        ifo=ifo,
        strain_data=strain_data,
        savedir=savedir
    )

In [ ]:
# Create and run processes sequentially (for simplicity in the notebook).
get_coherence_volt(volt_chans['channel'], ifo, time_chosen, ht_data, savedir, dur)
# get_coherence_omc(omc_chans['channel'], ifo, time_chosen, ht_data, savedir, dur)
get_coherence_mic(mic_chans['channel'], ifo, time_chosen, ht_data, savedir, dur)

print("Coherence processing complete.")

In [ ]:
# Assume that run_coherence saves CSV files in a subdirectory named with the GPS time.
output_dir = os.path.join(savedir, f'{int(time_chosen)}')
print(output_dir)

vals = get_max_corr(output_dir, save=True)
print(vals)
if vals.empty:
    raise ValueError("No maximum correlation data found.")

In [ ]:
# Define a simple grouping function (you may adjust it as needed).
def give_group(a):
    # For example, group by the second component of the channel name.
    return a.split('_')[1]

vals['group'] = vals['channel'].apply(give_group)
print(vals['group'])

In [ ]:
# Create a scatter plot of maximum correlation vs. frequency.
fig = px.scatter(
    vals, 
    x="frequency", 
    y="max_correlation",
    hover_data=['channel'], 
    color="group",
    labels={"max_correlation": "Max Correlation", "frequency": "Frequency [Hz]"}
)

# Update layout: increase title and legend fonts.
fig.update_layout(
    title=dict(
        text="Max Coherence during {} -- {}".format(str(time_chosen), str(time_chosen + dur)),
        font=dict(family="Courier New, monospace", size=28, color="Blue")
    ),
    legend=dict(font=dict(size=20))
)

# Increase the scatter marker size.
fig.update_traces(marker=dict(size=20, opacity=0.8))

# Display the plot in the notebook.
fig.show()

In [ ]:
# Write the vals DataFrame to a CSV file.
vals_output_file = os.path.join(savedir, f'vals_output_{int(time_chosen)}.csv')
vals.to_csv(vals_output_file, index=False)
print("vals saved to:", vals_output_file)

In [ ]:
# Now read the CSV file back in and display its first few rows.
vals_read = pd.read_csv(vals_output_file)
print("Contents of the saved vals file:")
display(vals_read.head())